In [1]:
import yfinance as yf

import pandas as pd
import numpy as np

from ipywidgets import interact, interact_manual
import ipywidgets as widgets

from datetime import date, timedelta
import os
import os.path
import time

In [2]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as mfm

%matplotlib inline

font_path = "/usr/share/fonts/opentype/cff/OTF/TraditionalChinese/SourceHanSerifTC-Regular.otf"
prop = mfm.FontProperties(fname=font_path) # find this font

In [3]:
data_dir = "data"

# Input Prediction

In [4]:
def _get_prices_remote(ticker, file_name):
    print(f"Fetching {ticker} from yfinance...")
    
    yf_ticker = yf.Ticker(ticker)
    prices = yf_ticker.history(period="max")
    
    prices.reset_index().to_csv(file_name, index=False)
    
    return prices

In [5]:
def get_prices(ticker):
    file_name = f"{data_dir}/{ticker}.csv"
    
    if not os.path.exists(file_name):
        return _get_prices_remote(ticker, file_name)

    now = time.time()
    stat = os.stat(file_name)
    if (now - stat.st_mtime) > (60 * 60 * 24):
        return _get_prices_remote(ticker, file_name)
    
    prices = pd.read_csv(file_name)
    prices["Date"] = pd.to_datetime(prices["Date"])
    prices = prices.set_index("Date")
    
        
    return prices

In [45]:
get_prices("0001.HK").index

DatetimeIndex(['2000-01-04', '2000-01-05', '2000-01-06', '2000-01-07',
               '2000-01-10', '2000-01-11', '2000-01-12', '2000-01-13',
               '2000-01-14', '2000-01-17',
               ...
               '2020-03-09', '2020-03-10', '2020-03-11', '2020-03-12',
               '2020-03-13', '2020-03-16', '2020-03-17', '2020-03-18',
               '2020-03-19', '2020-03-20'],
              dtype='datetime64[ns]', name='Date', length=5048, freq=None)

In [120]:
def get_results(file_name):
    cols = (("prediction_date", "datetime64[ns]", True),
            ("author", str, True),
            ("ticker", str, True),
            ("upper_target_price", float, False),
            ("low_target_price", float, False),
            ("stop_loss_price", float, False),
            ("result", str, False))

    index = list((col[0] for col in filter(lambda cols: cols[2], cols)))
    
    if not os.path.exists(file_name):
        df = pd.DataFrame()
        for c, t, i in cols:
            df[c] = pd.Series(dtype=t)

        df = df.set_index(index)
        return df

    df = pd.read_csv(file_name)
    for c, t, i in cols:
        df[c] = df[c].astype(t)
    
    df = df.set_index(index)
    return df

In [133]:
def save_prediction_result(ticker, prediction_date, upper_target_price, lower_target_price, stop_loss_price,
                           title, author, result):
    file_name = f"{data_dir}/prediction_results.csv"
    results = get_results(file_name)

    results.loc[(prediction_date, author, ticker)] = (upper_target_price, lower_target_price, stop_loss_price, result)
    
    results.to_csv(file_name)
    return results

In [130]:
ticker = "1347.HK"

prediction_date = date(2019,7, 2)
upper_target_price = 18
lower_target_price = 17.3
stop_loss_price = 14.3

title=u"晶片股調整夠 華虹現價買入"
author=u"邱古奇"

save_prediction_result(ticker, prediction_date, 
                       upper_target_price, lower_target_price, stop_loss_price, 
                       title, author, "Profit")

                                upper_target_price  low_target_price  \
prediction_date author ticker                                          
2019-07-02      邱古奇    1347.HK                18.0              17.3   

                                stop_loss_price  result  
prediction_date author ticker                            
2019-07-02      邱古奇    1347.HK             14.3  Profit  


,,,upper_target_price,low_target_price,stop_loss_price,result
prediction_date,author,ticker,,,,
2019-07-02,邱古奇,1347.HK,18.0,17.3,14.3,Profit


In [14]:
def simulate_prediction(ticker, prediction_date, upper_target_price, lower_target_price, stop_loss_price, title, author):
    last_holding_date = prediction_date + timedelta(days=31)

    ceiling = lower_target_price
    floor = stop_loss_price

    prices = get_prices(ticker)
    for row in prices.itertuples(name="Price"):
        #if row.Index < prediction_date:
        if row.Index < last_holding_date:
            continue

        if row.Close >= ceiling:
            return {"result": "Profit", "date": row.Index, "price": row.Close}
            break

        if row.Close <= floor:
            return {"result": "Loss", "date": row.Index, "price": row.Close}
            break
            
    return {"result": "N/A", "date": None, "price": None}

        


In [138]:
def plot_prediction(ticker, prediction_date, upper_target_price, lower_target_price, stop_loss_price,
                    title, author):
    if not ticker:
        return
    
    prices = get_prices(ticker)
    
    data = prices["Close"]

    start_date = prediction_date - timedelta(days=30)
    start_date = date(start_date.year, start_date.month, 1)
    data = data[data.index >= np.datetime64(start_date)]

    #plt.xkcd()
    plt.figure(figsize=(15, 10))

    plt.axvline(prediction_date, label="prediction date", c="black")

    last_holding_date = prediction_date + timedelta(days=31)
    plt.axvline(last_holding_date, label="last day of holding", c="brown")

    if upper_target_price != 0.0:
        plt.axhline(upper_target_price, label="upper target price", c="green")
    
    if lower_target_price != 0.0:
        plt.axhline(lower_target_price, label="lower target price", c="limegreen")
    
    if stop_loss_price == 0.0:
        stop_loss_price = data.loc[prediction_date] * 0.9
    
    plt.axhline(stop_loss_price, label="stop price price", c="red")

    result = simulate_prediction(ticker, prediction_date, upper_target_price, lower_target_price, stop_loss_price,
                                 title, author)
    
    save_prediction_result(ticker, prediction_date, 
                       upper_target_price, lower_target_price, stop_loss_price, 
                       title, author, result["result"])
    
    if result["date"] is not None:
        #plt.text(x=result["date"], y=result["price"], s=result["result"])
        plt.annotate(s=result["result"], xy=(result["date"], result["price"]),
                     xytext=(result["date"], result["price"]*1.2),
                     arrowprops=dict(arrowstyle="->"),
                    fontsize=20)
    
    prop = mfm.FontProperties(fname=font_path) # find this font

    plt.suptitle(title, fontproperties=prop, fontsize=30)
    plt.title(author, fontproperties=prop, fontsize=20)
    plt.legend()
    
    data.plot(legend=False, grid=True)

In [139]:
ticker = "1347.HK"

prediction_date = date(2019,7, 2)
upper_target_price = 18
lower_target_price = 17.3
stop_loss_price = 14.3

title=u"晶片股調整夠 華虹現價買入"
author=u"邱古奇"


#plot_prediction(ticker, prediction_date, upper_target_price, lower_target_price, stop_loss_price, title, author)
    
interact_manual(plot_prediction,
                ticker="", 
                prediction_date=widgets.DatePicker(description="Prediction Date"),
                upper_target_price=widgets.FloatText(description="Upper Target"),
                lower_target_price=widgets.FloatText(description="Lower Target"),
                stop_loss_price=widgets.FloatText(description="Stop Loss"),
                title="",
                author="邱古奇")

interactive(children=(Text(value='', description='ticker'), DatePicker(value=None, description='Prediction Dat…

<function __main__.plot_prediction(ticker, prediction_date, upper_target_price, lower_target_price, stop_loss_price, title, author)>